In [59]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
# from keras.models import Sequential
# from keras.layers import Input, Dense, Dropout
# from keras.optimizers import Adam
from sklearn.metrics import log_loss
import matplotlib.pyplot as  plt
from sklearn import tree                                                        

SPLIT_SEED = 1
MENS = 'mens'
WOMENS = 'womens'
TEST = 'test'
TRAIN = 'train'

# Columns in Data With Categorical Values- Must LabelEncode them
categorical_cols = ['hitpoint', 'outside.sideline', 'outside.baseline', 'same.side', 
                    'previous.hitpoint', 'server.is.impact.player', 'outcome']

# Columns in the Data That Should Be Scaled
scaled_data = ['serve', 'rally', 'speed', 'net.clearance', 'distance.from.sideline', 'depth', 'player.distance.travelled', 
               'player.impact.depth', 'player.impact.distance.from.center', 'player.depth', 
               'player.distance.from.center', 'previous.speed', 'previous.net.clearance', 
               'previous.distance.from.sideline', 'previous.depth', 'opponent.depth', 
               'opponent.distance.from.center', 'previous.time.to.net']
               



In [64]:
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
import pymysql

# Load Data

host = "gtbootcamp.mysql.database.azure.com"
user = "harish@gtbootcamp"
password = "JsOverload!23"
db = "tennis_db"
con = pymysql.connect(host=host, user=user, password=password, db=db, cursorclass=pymysql.cursors.
                                   DictCursor)
cur = con.cursor()
mens_train = pd.read_csv('tennis_data/mens_train_file.csv')
mens_test = pd.read_csv("tennis_data/mens_test_file.csv")
# print((mens_test.head()))
# Encode Categorical Data
def encode(train, test):
    # Retain All LabelEncoder as a dictionary
    d = defaultdict(LabelEncoder)

    # Encode all the columns
    train[categorical_cols] = train[categorical_cols].apply(lambda x: d[x.name].fit_transform(x))
#     test[categorical_cols] = train[categorical_cols].apply(lambda x: d[x.name].fit_transform(x))
    test_ids = test['id']
    # Inverse the encoding
    # data.apply(lambda x: d[x.name].inverse_transform(x))
    
    # Using dictionary d to label future data
    temp = deepcopy(categorical_cols)
    temp.remove('outcome')
    e = deepcopy(d)
    del e['outcome']
#     for key in e.keys():
#         print(key)
#         print(e[key].classes_)
#     print(temp)
    
    test[temp] = test[temp].apply(lambda x: e[x.name].transform(x))
# #     print(d['hitpoint'].classes_)
    train = train.drop(['gender', 'id', 'train'], axis=1)
    test = test.drop(['gender', 'id', 'outcome', 'train'], axis=1)
    
    return train, test, test_ids, d

# print(train['serve'])
mens_train, mens_test, test_ids , d = encode(mens_train, mens_test)
# print(mens_test_X.head())


In [65]:
from sklearn.model_selection import train_test_split

mens_train_X, val_mens_X = train_test_split(mens_train, test_size=0.2, shuffle=True)
X_train = mens_train_X.loc[:, mens_train_X.columns != 'outcome']
y_train = mens_train_X['outcome']
X_val = val_mens_X.loc[:, val_mens_X.columns != 'outcome']
y_val = val_mens_X['outcome']

In [67]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_val)
    acc = accuracy_score(y_val, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_val)
    ll = log_loss(y_val, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 67.0000%
Log Loss: 5.331372859362093
SVC
****Results****
Accuracy: 42.7000%
Log Loss: 0.8246950543284836
DecisionTreeClassifier
****Results****
Accuracy: 77.1000%
Log Loss: 7.909379794434547
RandomForestClassifier
****Results****
Accuracy: 83.2000%
Log Loss: 0.7518943599769671
